In [23]:
import pandas as pd
from pennylane import numpy as np
import math
import pennylane as qml
import random
import tensorflow as tf
import time

In [24]:
# Read data
df = pd.read_csv('dataset_FTSE_prediction_v1.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df=df.drop(columns=['ARIMA'])
df=df[800:]

In [25]:
target = "Close"
features = list(df.columns.difference(["Close"]))
features

['Adj Close',
 'BBB_20_2.0',
 'BBL_20_2.0',
 'BBM_20_2.0',
 'BBP_20_2.0',
 'BBU_20_2.0',
 'CUMLOGRET_1',
 'Close_copy',
 'Crude Futures',
 'EMA_10',
 'EMA_20',
 'EMA_50',
 'FT3',
 'FT6',
 'FT9',
 'GBP USD ',
 'Gold in USD',
 'MACD_12_26_9',
 'MACDh_12_26_9',
 'MACDs_12_26_9',
 'SMA_10',
 'SMA_20',
 'SMA_50',
 'Volume']

In [26]:
len(features)

24

## Data Processing

In [27]:
random.seed(42)
# close_min=min(df['Close'].values)
# close_max=max(df['Close'].values)
# normalized_df=(df-df.min())/(df.max()-df.min())

In [28]:
target_mean = df[target].mean()
target_stdev = df[target].std()

for c in df.columns:
    mean = df[c].mean()
    stdev = df[c].std()

    df[c] = (df[c] - mean) / stdev

In [29]:
df.head()

,Adj Close,BBB_20_2.0,BBL_20_2.0,BBM_20_2.0,BBP_20_2.0,BBU_20_2.0,CUMLOGRET_1,Close,Crude Futures,EMA_10,...,GBP USD,Gold in USD,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,SMA_10,SMA_20,SMA_50,Volume,Close_copy
800,-0.664382,-0.518756,-0.212867,-0.376660,-2.096023,-0.541526,-0.618721,-0.664382,1.732245,-0.477768,...,2.765372,-0.923083,-0.285174,-0.957418,0.029832,-0.442130,-0.376660,-0.460099,0.479939,-0.664382
801,-0.646763,-0.405198,-0.255201,-0.392525,-1.648446,-0.526270,-0.600796,-0.646763,1.732245,-0.510207,...,2.765372,-0.947383,-0.387140,-1.015113,-0.059456,-0.471115,-0.392525,-0.463963,-0.746811,-0.646763
802,-0.651849,-0.315221,-0.292326,-0.409164,-1.448674,-0.518568,-0.605967,-0.651849,1.742954,-0.537690,...,2.671345,-0.935746,-0.466173,-1.005142,-0.147868,-0.500101,-0.409164,-0.467120,-0.835320,-0.651849
803,-0.695260,-0.219031,-0.333231,-0.428391,-1.458203,-0.511942,-0.650199,-0.695260,1.607479,-0.568214,...,2.671345,-0.959704,-0.550693,-1.010594,-0.236759,-0.531275,-0.428391,-0.473782,0.314693,-0.695260
804,-0.813687,-0.091393,-0.390878,-0.457822,-1.732959,-0.507468,-0.771680,-0.813687,1.580170,-0.615116,...,2.671345,-0.945671,-0.686218,-1.139791,-0.336992,-0.572362,-0.457822,-0.483073,-0.020605,-0.813687


In [30]:
# Data for regression model
sequence_length_reg = window_reg =20 # Number of time steps that will be inputted into LSTM network

def create_dataset_reg(dataset,target,features, lookback, window):
    X_store, y_store = [], []
    for i in range(0,len(dataset)-lookback-window,5):
        X = (dataset[features].values)[i:i+lookback]
        y = (dataset[target].values)[i+lookback-1:i+lookback+window-1]
        X_store.append(X)
        y_store.append(y)
    return X_store,y_store

X_reg,y_reg = create_dataset_reg(
    df,
    target=target,
    features=features,
    lookback=sequence_length_reg, window=window_reg)

print(np.array(X_reg).shape, np.array(y_reg).shape)

(460, 20, 24) (460, 20)


In [31]:
indices = tf.range(start=0, limit=tf.shape(X_reg)[0])
shuffled_indices = tf.random.shuffle(indices)
X_reg = tf.gather(X_reg, shuffled_indices)
y_reg = tf.gather(y_reg, shuffled_indices)

In [32]:
# Train test split
m=np.array(X_reg).shape[0] # get first dimension
size=int(m*0.9)
X_train_reg=X_reg[:size]
y_train_reg=y_reg[:size]
X_test_reg=X_reg[size:]
y_test_reg=y_reg[size:]
print('Shape of train is',np.array(X_train_reg).shape,'and',np.array(y_train_reg).shape)
print('Shape os test is',np.array(X_test_reg).shape,'and',np.array(y_test_reg).shape)

Shape of train is (414, 20, 24) and (414, 20)
Shape os test is (46, 20, 24) and (46, 20)


In [33]:
X_train_reg_=np.array([inner for outer in X_train_reg for inner in outer])
print(X_train_reg_.shape)
y_train_reg_=np.array([inner for outer in y_train_reg for inner in outer])
print(y_train_reg_.shape)

(8280, 24)
(8280,)


In [34]:
X_test_reg_=np.array([inner for outer in X_test_reg for inner in outer])
print(X_test_reg_.shape)
y_test_reg_=np.array([inner for outer in y_test_reg for inner in outer])
print(y_test_reg_.shape)

(920, 24)
(920,)


In [35]:
# Prepare check data
X_check_reg=(df[features].values)[len(df)-window_reg-sequence_length_reg+1:len(df)- window_reg+1]
y_check_reg=(df[target].values)[len(df)-window_reg:len(df)]

In [36]:
# Python Program to find the factors of a number

# This function computes the factor of the argument passed
def print_factors(x):
    print("The factors of",x,"are:")
    for i in range(1, x + 1):
        if x % i == 0:
            print(i)
num = 8280
print_factors(num)

The factors of 8280 are:
1
2
3
4
5
6
8
9
10
12
15
18
20
23
24
30
36
40
45
46
60
69
72
90
92
115
120
138
180
184
207
230
276
345
360
414
460
552
690
828
920
1035
1380
1656
2070
2760
4140
8280


# QNN

In [44]:
nqubits = 8
device = qml.device('default.qubit', wires=nqubits)
# Define QNode
@qml.qnode(device)
def quantum_circuit(inputs, params):
    for p1,p2,p3,p4,p5,p6,p7,p8 in zip(params[:4],params[4:8],params[8:12],params[12:16],params[16:20],params[20:24]
                                ,params[24:28],params[28:32]):
        qml.Rot(*inputs[:3], wires=0)
        qml.Rot(*inputs[3:6], wires=1)
        qml.Rot(*inputs[6:9], wires=2)
        qml.Rot(*inputs[9:12], wires=3)
        qml.Rot(*inputs[12:15], wires=4)
        qml.Rot(*inputs[15:18], wires=5)
        qml.Rot(*inputs[18:21], wires=6)
        qml.Rot(*inputs[21:24], wires=7)
        qml.Rot(*p1, wires=0)
        qml.Rot(*p2, wires=1)
        qml.Rot(*p3, wires=2)
        qml.Rot(*p4, wires=3)
        qml.Rot(*p5, wires=4)
        qml.Rot(*p6, wires=5)
        qml.Rot(*p7, wires=6)
        qml.Rot(*p8, wires=7)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4]) 
        qml.CNOT(wires=[4, 5])
        qml.CNOT(wires=[5, 6]) 
        qml.CNOT(wires=[6, 7])
        qml.CNOT(wires=[7, 0]) 
    return [qml.expval(qml.PauliZ(0))]

In [45]:
def loss_func(params):
    predictions = [np.add(params[32][2],np.multiply(params[32][1],np.add(params[32][0],quantum_circuit(x, params)))) 
                   for x in X_train_reg_]
    predictions=np.array([i for i in predictions])
    total_losses=np.mean((y_train_reg_ - predictions) ** 2)
    return total_losses

In [46]:
def cost_fn(params,**kwargs):
    z=kwargs['batch']
    predictions = [np.add(params[32][2],np.multiply(params[32][1],np.add(params[32][0],quantum_circuit(x, params)))) 
                   for x in X_train_reg_[z*batch_size: (z+1)*batch_size]]
    predictions=np.array([i for i in predictions])
    cost=np.mean((y_train_reg_[z*batch_size: (z+1)*batch_size] - predictions) ** 2)
    return cost

In [47]:
opt = qml.AdamOptimizer(stepsize=0.005)

params=np.random.normal(-0.05, 0.05, size=(33, 3),requires_grad = True)

In [48]:
len(params)

33

In [57]:
batch_size=36 # Batch size
batch_list=list(range(0,int(len(X_train_reg_)/batch_size)))

In [55]:
batch_=list(range(len(batch_list)))

In [ ]:
for i in range (100):
    batch_=list(range(len(batch_list)))
    while len(batch_)>0:
        j= random.choice(batch_)
        batch_.remove(j)
        params, prev_cost = opt.step_and_cost(cost_fn,params,batch=j)
    if i%5==0:
        print(f'Step = {i} Cost = {loss_func(params)}')

Step = 0 Cost = 1.0173695711798056
Step = 5 Cost = 1.0360320105252057
Step = 10 Cost = 1.0817518623039213


In [ ]:
params

In [ ]:
# Make pred for check data
def pred_(X,params):
    predict_ = [np.add(params[32][2],np.multiply(params[32][1],np.add(params[32][0],quantum_circuit(x, params)))) 
                for x in X]
    return predict_

In [ ]:
def mae_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += abs(predicted[i] - actual[i])
    return sum_error/len(actual)

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += (predicted[i] - actual[i])**2
    return (sum_error/len(actual))**0.5

In [ ]:
# Predict the price
y_test_pred_ann = pred_(X_test_reg_,params) 

In [ ]:
y_test_pred_ann_=[]
for i in y_test_pred_ann:
    y_test_pred_ann_.append(i.numpy())

In [ ]:
mae_test_ann = mae_metric(y_test_reg_* target_stdev + target_mean, np.array(y_test_pred_ann_)* target_stdev + target_mean)
print('MAE test: ',mae_test_ann)
rmse_test_ann = rmse_metric(y_test_reg_* target_stdev + target_mean, np.array(y_test_pred_ann_)* target_stdev + target_mean)
print('RMSE test: ',rmse_test_ann)

In [ ]:
# Predict the model 
y_check_pred_ann = pred_(X_check_reg,params) 

In [ ]:
y_check_pred_ann_=[]
for i in y_check_pred_ann:
    y_check_pred_ann_.append(i[0].numpy())

In [ ]:
mae_check_ann = mae_metric(y_check_reg* target_stdev + target_mean , np.array(y_check_pred_ann_)*target_stdev + target_mean)
print('MAE test: ',mae_check_ann)
rmse_check_ann = rmse_metric(y_check_reg* target_stdev + target_mean, np.array(y_check_pred_ann_)*target_stdev + target_mean)
print('RMSE test: ',rmse_check_ann)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(y_check_reg* target_stdev + target_mean, label = "Real",marker='o')
plt.plot(np.array(y_check_pred_ann_)* target_stdev + target_mean, label = "Quantum ANN",marker='^')
plt.ylabel('Stock Price')
plt.xlabel('Days')
plt.legend()
plt.show()